Copyright 2021-2022 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of Cybertron package.

The Cybertron is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Cybertron tutorial 04: Read parameters and hyperparameters from checkpoint file
                       and to use test dataset with scale and shift.

In [1]:
import sys
import numpy as np
from mindspore import nn
from mindspore import context
from mindspore import dataset as ds
from mindspore.train import Model
from mindspore.train import load_checkpoint

from mindsponge.data import load_hyperparam

from cybertron import Cybertron
from cybertron.train import MAE
from cybertron.train import WithLabelEvalCell

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

[WARNING] ME(14474:140502773348160,MainProcess):2022-08-10-17:32:19.847.768 [mindspore/run_check/_check_version.py:137] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
ckpt_file = 'Tutorial_C03/Tutorial_C03_MolCT-best.ckpt'
hyper_param = load_hyperparam(ckpt_file)

In [3]:
net = Cybertron(hyper_param=hyper_param)
load_checkpoint(ckpt_file, net)

{'model.atom_embedding.embedding_table': Parameter (name=model.atom_embedding.embedding_table, shape=(64, 128), dtype=Float32, requires_grad=True),
 'model.dis_filter.linear.weight': Parameter (name=model.dis_filter.linear.weight, shape=(128, 64), dtype=Float32, requires_grad=True),
 'model.dis_filter.linear.bias': Parameter (name=model.dis_filter.linear.bias, shape=(128,), dtype=Float32, requires_grad=True),
 'model.dis_filter.residual.nonlinear.mlp.0.weight': Parameter (name=model.dis_filter.residual.nonlinear.mlp.0.weight, shape=(128, 128), dtype=Float32, requires_grad=True),
 'model.dis_filter.residual.nonlinear.mlp.0.bias': Parameter (name=model.dis_filter.residual.nonlinear.mlp.0.bias, shape=(128,), dtype=Float32, requires_grad=True),
 'model.dis_filter.residual.nonlinear.mlp.1.weight': Parameter (name=model.dis_filter.residual.nonlinear.mlp.1.weight, shape=(128, 128), dtype=Float32, requires_grad=True),
 'model.dis_filter.residual.nonlinear.mlp.1.bias': Parameter (name=model.dis

In [4]:
idx = [7]  # U0

test_file = sys.path[0] + '/dataset_qm9_origin_testset_1024.npz'
test_data = np.load(test_file)

scale = test_data['scale'][idx]
shift = test_data['shift'][idx]
ref = test_data['type_ref'][:, idx]

net.set_scaleshift(scale=scale, shift=shift, type_ref=ref, unit='kj/mol')
net.print_info()

Cybertron Engine, Ride-on!
--------------------------------------------------------------------------------
    Length unit: nm
    Input unit scale: 1
--------------------------------------------------------------------------------
    Deep molecular model:  MolCT
--------------------------------------------------------------------------------
       Length unit: nm
       Atom embedding size: 64
       Cutoff distance: 1.0 nm
       Radical basis function (RBF): LogGaussianBasis
          Minimum distance: 0.04 nm
          Maximum distance: 1.0 nm
          Reference distance: 1.0 nm
          Log Gaussian begin: -3.218876
          Log Gaussian end: 0.006724119
          Interval for log Gaussian: 0.0512
          Sigma for log gaussian: 0.3
          Number of basis functions: 64
          Rescale the range of RBF to (-1,1).
       Calculate distance: Yes
       Calculate bond: No
       Feature dimension: 128
-----------------------------------------------------------------------

In [5]:
ds_test = ds.NumpySlicesDataset(
    {'R': test_data['R'], 'Z': test_data['Z'], 'E': test_data['E'][:, idx]}, shuffle=False)
ds_test = ds_test.batch(1024)
ds_test = ds_test.repeat(1)

In [6]:
eval_network = WithLabelEvalCell('RZE', net, nn.MAELoss())

WithLabelEvalCell with input type: RZE


In [7]:
eval_mae = 'EvalMAE'
atom_mae = 'AtomMAE'
model = Model(net, eval_network=eval_network, metrics={eval_mae: MAE(
    [1, 2]), atom_mae: MAE([1, 2, 3], averaged_by_atoms=True)})

In [8]:
print('Test dataset:')
eval_metrics = model.eval(ds_test, dataset_sink_mode=False)
info = ''
for k, v in eval_metrics.items():
    info += k
    info += ': '
    info += str(v)
    info += ', '
print(info)

Test dataset:
EvalMAE: 43.42962646484375, AtomMAE: 2.5831985473632812, 


In [9]:
outdir = 'Tutorial_C04'
scaled_ckpt = outdir + '_' + net.model_name + '.ckpt'
net.save_checkpoint(scaled_ckpt, outdir)

Cybertron<
  (model): MolCT<
    (activation): Swish<
      (sigmoid): Sigmoid<>
      >
    (atom_embedding): Embedding<vocab_size=64, embedding_size=128, use_one_hot=True, embedding_table=Parameter (name=model.atom_embedding.embedding_table, shape=(64, 128), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>
    (cutoff_fn): SmoothCutoff<>
    (rbf): LogGaussianBasis<>
    (dis_filter): ResFilter<
      (linear): Dense<input_channels=64, output_channels=128, has_bias=True>
      (residual): Residual<
        (nonlinear): MLP<
          (mlp): SequentialCell<
            (0): Dense<
              input_channels=128, output_channels=128, has_bias=True, activation=Swish<>
              (activation): Swish<
                (sigmoid): Sigmoid<>
                >
              >
            (1): Dense<input_channels=128, output_channels=128, has_bias=True>
            >
          >
        >
      >
    (interactions): CellList<
      (0): NeuralInteractionUnit<
        (

In [10]:
net2 = Cybertron(hyper_param=load_hyperparam(outdir+'/'+scaled_ckpt))
net2.print_info()

Cybertron Engine, Ride-on!
--------------------------------------------------------------------------------
    Length unit: nm
    Input unit scale: 1
--------------------------------------------------------------------------------
    Deep molecular model:  MolCT
--------------------------------------------------------------------------------
       Length unit: nm
       Atom embedding size: 64
       Cutoff distance: 1.0 nm
       Radical basis function (RBF): LogGaussianBasis
          Minimum distance: 0.04 nm
          Maximum distance: 1.0 nm
          Reference distance: 1.0 nm
          Log Gaussian begin: -3.218876
          Log Gaussian end: 0.006724119
          Interval for log Gaussian: 0.0512
          Sigma for log gaussian: 0.3
          Number of basis functions: 64
          Rescale the range of RBF to (-1,1).
       Calculate distance: Yes
       Calculate bond: No
       Feature dimension: 128
-----------------------------------------------------------------------